<a href="https://colab.research.google.com/github/MarlaS02/comp215/blob/Practice/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name(s):
#### Date:

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [1]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [2]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-04-12 <class 'datetime.date'>
Today is: 2025-04-12


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [37]:
API_KEY = 'o2bsUdeZ6NhRHgHBxjC19Mxwj9QmkaykdtzGymNg'  # substitute your API key here

#get today's date (as a string)
today = str(datetime.date.today())

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})    #this is the same as response = requests.get(url)

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)
pprint(data)
# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count'] #this is the number of results
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:3])

{'element_count': 9,
 'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2025-04-14&end_date=2025-04-14&detailed=false&api_key=o2bsUdeZ6NhRHgHBxjC19Mxwj9QmkaykdtzGymNg',
           'prev': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2025-04-12&end_date=2025-04-12&detailed=false&api_key=o2bsUdeZ6NhRHgHBxjC19Mxwj9QmkaykdtzGymNg',
           'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2025-04-13&end_date=2025-04-13&detailed=false&api_key=o2bsUdeZ6NhRHgHBxjC19Mxwj9QmkaykdtzGymNg'},
 'near_earth_objects': {'2025-04-13': [{'absolute_magnitude_h': 21.45,
                                        'close_approach_data': [{'close_approach_date': '2025-04-13',
                                                                 'close_approach_date_full': '2025-Apr-13 '
                                                                                             '07:14',
                                                                 'epoch_date_close_approach': 174452

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [12]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')
pprint(hazards)

1 potentially hazardous asteroids identified.
[{'absolute_magnitude_h': 18.9,
  'close_approach_data': [{'close_approach_date': '2025-04-12',
                           'close_approach_date_full': '2025-Apr-12 03:55',
                           'epoch_date_close_approach': 1744430100000,
                           'miss_distance': {'astronomical': '0.0277410749',
                                             'kilometers': '4150005.716550463',
                                             'lunar': '10.7912781361',
                                             'miles': '2578693.9766358694'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '50241.2903200026',
                                                 'kilometers_per_second': '13.9559139778',
                                                 'miles_per_hour': '31217.9839768927'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 3236.1230737181,
   

### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [7]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 18.9,
 'close_approach_data': [{'close_approach_date': '1910-04-11',
                          'close_approach_date_full': '1910-Apr-11 18:20',
                          'epoch_date_close_approach': -1884750000000,
                          'miss_distance': {'astronomical': '0.0665240723',
                                            'kilometers': '9951859.519806001',
                                            'lunar': '25.8778641247',
                                            'miles': '6183798.7590487738'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '50089.6627831494',
                                                'kilometers_per_second': '13.9137952175',
                                                'miles_per_hour': '31123.7685221186'}},
                         {'close_approach_date': '1923-04-12',
                          'close_approach_date_full': '1923-Apr-12 06:12',


## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [8]:
# Ex. 1 your code here
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroids identified')

1 potentially hazardous asteroids identified




## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"


Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [40]:
# Ex. 2 your code here
def closest_approach(close_approach_data):
  closest = close_approach_data[0]
  for item in close_approach_data:
    if item[0]['orbiting_body'] == 'Earth':
      if item[0]['miss_distance']['kilometers'] < closest[0]['miss_distance']['kilometers']:
           closest = item
  return (closest[0]['close_approach_date'], closest[0]['miss_distance']['kilometers'])


closest_approach_data = []
for item in neos:
  closest_approach_data.append(item['close_approach_data'])
#pprint(neos)
#pprint(closest_approach_data)
print(closest_approach(closest_approach_data))

('2025-04-13', '22671461.545621922')


## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [48]:
# Ex. 3 (challenge) your code here
import datetime

end_date = str(datetime.date.today())
start_date = str(datetime.date.today() - datetime.timedelta(days=7))

url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'

response = requests.get(url)
data = json.loads(response.text)

neos = data['near_earth_objects']

approach_distance_list = []

for i in range(len(neos)):
  date = str(datetime.date.today() - datetime.timedelta(days=i))
  data = neos[date]
  closest_approach_data = []
  for item in data:
    closest_approach_data.append(item['close_approach_data'])
    id = item['id']
    approach_date, approach_distance = closest_approach(closest_approach_data)
    approach_distance_list.append((id, approach_distance, approach_date))
    print(f'The NEO with ID: {id} will come {approach_distance} km close to Earth on {approach_date}')

closest = approach_distance_list[0]
for i in range(len(approach_distance_list)-1):
  if approach_distance_list[i+1][1] < closest[1]:
    closest = approach_distance_list[i+1]
id, approach_distance, approach_date = closest
print(f'The Neo with the closest approach to Earth this week has ID {id} and will pass us with a distance of {approach_distance} km on {approach_date}')
#pprint(neos)

The NEO with ID: 3703885 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 54016746 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 54134936 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 54340154 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 54380881 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 54497462 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 54525113 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 54526038 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 2784199 will come 22671461.545621922 km close to Earth on 2025-04-13
The NEO with ID: 2527977 will come 74405925.657814076 km close to Earth on 2025-04-12
The NEO with ID: 3441092 will come 71776455.29618177 km close to Earth on 2025-04-12
The NEO with ID: 3713935 will come 5610112.41810